# Hiring Test Scratch Notebook

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# AVG = (H/AB)
# OBP = (H + BB + HBP) / (AB + BB + HBP + SF)
# SLG = (TB/AB)
# OPS = OBP + SLG

In [3]:
# Import raw data
raw_df = pd.read_csv('../data/raw/pitchdata.csv')
print(raw_df.shape)

(20066, 18)


In [4]:
raw_df.head()

,GameId,PitcherId,HitterId,PitcherSide,HitterSide,PrimaryEvent,PitcherTeamId,HitterTeamId,PA,AB,H,2B,3B,HR,TB,BB,SF,HBP
0,2016/09/15/clemlb-chamlb-1,448306,424825,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
1,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,BHA,145,114,0,0,0,0,0,0,0,0,0,0
2,2016/09/15/clemlb-chamlb-1,448306,543401,R,L,GO,145,114,1,1,0,0,0,0,0,0,0,0
3,2016/09/15/clemlb-chamlb-1,448306,608070,R,L,FO,145,114,1,1,0,0,0,0,0,0,0,0
4,2016/09/15/clemlb-chamlb-1,605182,594809,R,L,KS,114,145,1,1,0,0,0,0,0,0,0,0


In [5]:
# Import reference data
ref_df = pd.read_csv('../data/reference/combinations.txt')
print(ref_df.shape)

(32, 3)


In [13]:
ref_df

,Stat,Subject,Split
0,AVG,HitterId,vs RHP
1,OBP,HitterId,vs RHP
2,SLG,HitterId,vs RHP
3,OPS,HitterId,vs RHP
4,AVG,HitterId,vs LHP
5,OBP,HitterId,vs LHP
6,SLG,HitterId,vs LHP
7,OPS,HitterId,vs LHP
8,AVG,HitterTeamId,vs RHP
9,OBP,HitterTeamId,vs RHP


In [34]:
team_ids = raw_df.PitcherTeamId.unique()
team_ids

array([145, 114, 109, 119, 112, 158, 116, 142, 111, 147, 118, 133, 143,
       134, 137, 138, 110, 139, 108, 141, 136, 117, 146, 121, 140, 113,
       115, 135, 144, 120])

In [39]:
pitcher_ids = raw_df.PitcherId.unique()
print('pitchers', len(pitcher_ids))
hitter_ids = raw_df.HitterId.unique()
print('hitters', len(hitter_ids))

pitchers 532
hitters 624


In [92]:
def calc_avg(df):
    """Calculate batting average: hits divided by at bats (H/AB)"""
    avg = sum(df.H)/sum(df.AB)
    return avg
    
def calc_obp(df):
    """Calculate on-base percentage: times reached base (H + BB + HBP) divided by at bats plus 
    walks plus hit by pitch plus sacrifice flies (AB + BB + HBP + SF)"""
    obp = (sum(df.H) + sum(df.BB) + sum(df.HBP))/(sum(df.AB) + sum(df.BB) + sum(df.HBP) + sum(df.SF))
    return obp
    
def calc_slg(df):
    """Calculate slugging average: total bases achieved on hits divided by at-bats (TB/AB)"""
    slg = (sum(df.TB)/sum(df.AB))
    return slg

def calc_ops(obp, slg):
    """On-base plus slugging: on-base percentage plus slugging average"""
    ops = obp + slg
    return ops

def calc_subject_stats(df, results, subject_id, subject_name, split):
    """Calculate AVG, OBP, SLG, and OPS stats for given subject and add it to results array"""
    avg = calc_avg(df)
    results.append([subject_id, "AVG", split, subject_name, avg.round(3)])
    obp = calc_obp(df)
    results.append([subject_id, "OBP", split, subject_name, obp.round(3)])
    slg = calc_slg(df)
    results.append([subject_id, "SLG", split, subject_name, slg.round(3)])
    ops = calc_ops(obp, slg)
    results.append([subject_id, "OPS", split, subject_name, ops.round(3)])
    return results
   

def calc_pitcher_team_stats(raw_df, results, team_ids):
    """Extract relevant observations from dataframe for pitcher team subjects, run stat calculations, 
        and add to given results array."""
    for team_id in team_ids:
        team_df = raw_df[raw_df['PitcherTeamId'] == team_id]
        if sum(team_df.PA) >= 25:
            for side in ['R', 'L']:
                split = "vs " + side + "HH"
                df = team_df[team_df.HitterSide == side]
                results = calc_subject_stats(df, results, team_id, 'PitcherTeamId', split)
    return results

def calc_hitter_team_stats(raw_df, results, team_ids):
    """Extract relevant observations from dataframe for hitter team subjects, run stat calculations, 
        and add to given results array."""
    for team_id in team_ids:
        team_df = raw_df[raw_df['HitterTeamId'] == team_id]
        if sum(team_df.PA) >= 25:
            for side in ['R', 'L']:
                split = "vs " + side + "HP"
                df = team_df[team_df.PitcherSide == side]
                results = calc_subject_stats(df, results, team_id, 'HitterTeamId', split)
    return results

def calc_pitcher_player_stats(raw_df, results, pitcher_ids):
    """Extract relevant observations from dataframe for pitcher player subjects, run stat calculations, 
        and add to given results array."""
    for pitcher_id in pitcher_ids:
        pitcher_df = raw_df[raw_df['PitcherId'] == pitcher_id]
        for side in ['R', 'L']:
            split = "vs " + side + "HH"
            df = pitcher_df[pitcher_df.HitterSide == side]
            if sum(df.PA) >= 25:
                results = calc_subject_stats(df, results, pitcher_id, 'PitcherId', split)
    return results
                
def calc_hitter_player_stats(raw_df, results, hitter_ids):
    """Extract relevant observations from dataframe for hitter player subjects, run stat calculations, 
        and add to given results array."""
    for hitter_id in hitter_ids:
        hitter_df = raw_df[raw_df['HitterId'] == hitter_id]
        for side in ['R', 'L']:
            split = "vs " + side + "HP"
            df = hitter_df[hitter_df.PitcherSide == side]
            if sum(df.PA) >= 25:
                results = calc_subject_stats(df, results, hitter_id, 'HitterId', split)
    return results
    


In [93]:
results = []

results = calc_pitcher_team_stats(raw_df, results, team_ids)

results = calc_hitter_team_stats(raw_df, results, team_ids)

results = calc_pitcher_player_stats(raw_df, results, pitcher_ids)

results = calc_hitter_player_stats(raw_df, results, hitter_ids)


In [94]:
results_df = pd.DataFrame(results, columns=["SubjectId","Stat","Split","Subject","Value"])
sorted_results_df = results_df.sort_values(["SubjectId", "Stat", "Split"])
sorted_results_df

,SubjectId,Stat,Split,Subject,Value
148,108,AVG,vs LHH,PitcherTeamId,0.224
388,108,AVG,vs LHP,HitterTeamId,0.227
144,108,AVG,vs RHH,PitcherTeamId,0.272
384,108,AVG,vs RHP,HitterTeamId,0.247
149,108,OBP,vs LHH,PitcherTeamId,0.280
389,108,OBP,vs LHP,HitterTeamId,0.284
145,108,OBP,vs RHH,PitcherTeamId,0.346
385,108,OBP,vs RHP,HitterTeamId,0.333
151,108,OPS,vs LHH,PitcherTeamId,0.640
391,108,OPS,vs LHP,HitterTeamId,0.671


In [71]:
# Look at output
output_df = pd.read_csv('../data/reference/output.csv')
print(output_df.shape)

(2540, 5)


In [81]:
output_df

,SubjectId,Stat,Split,Subject,Value
0,108,AVG,vs LHH,PitcherTeamId,0.224
1,108,AVG,vs LHP,HitterTeamId,0.227
2,108,AVG,vs RHH,PitcherTeamId,0.272
3,108,AVG,vs RHP,HitterTeamId,0.247
4,108,OBP,vs LHH,PitcherTeamId,0.280
5,108,OBP,vs LHP,HitterTeamId,0.284
6,108,OBP,vs RHH,PitcherTeamId,0.346
7,108,OBP,vs RHP,HitterTeamId,0.333
8,108,OPS,vs LHH,PitcherTeamId,0.640
9,108,OPS,vs LHP,HitterTeamId,0.671
